## Webscraping Data

### Introduction:
Two webscraping operations were carried out on https://cars.com website in two batches:
1. Webscrape a total of 3100 URL for each car listed on the results page of the built-in search engine of the website.
2. Webscrape each car's brand/year and information on price, mileage, engine, transmission, interior and exterior colors, VIN, stock number and city/highway mpg.

In [1]:
# Import necessary libraries for webscraping.
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import json

In [ ]:
# cars.com search results page

### Creating the web scraping functions

In [2]:
# The function below executes the first webscrape operation to parse each car's URL
# For pagination, the url is split into two parts (urlpt1 and urlpt2) to change webpage number.
# ns parameter is a list used to create the page numbers for pagination

def link_list_v0(urlpt1,urlpt2, ns):
    list_o_urls = []
    for n in ns:
        url = urlpt1+str(n)+urlpt2 # complete URL stringed togather
        html1 = requests.get(url)
        soup1 = BeautifulSoup(html1.content, 'html.parser')
        url_ext = soup1.findAll('div',{'class':'shop-srp-listings__listing-container'}) # returns URL container
        time.sleep(1)
        for ext in url_ext: # loops through container to get the url extension
            list_o_urls.append('https://www.cars.com'+ext.find('a').attrs['href']) #appends it to empty list
            
    return list_o_urls
        

In [3]:
# Next is to define the function that parses the webpage of each used car on sale on the website
# using the URLs extracted from the previous step we proceeded to extract information relavent to the
# resale value of cars

In [4]:
def car_parser_v2(list_o_links):
    all_cars = []
    for link in list_o_links: #loop through the list of links
        try: # anticipating requests.get() errors
            page = requests.get(link, timeout=5)
            if page.status_code != 200:
                print(page.status_code)
        except requests.ConnectionError as e:
            print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
            print(str(e))
        except requests.Timeout as e:
            print("OOPS!! Timeout Error")
            print(str(e))
        except requests.RequestException as e:
            print("OOPS!! General Error")
            print(str(e))
        except KeyboardInterrupt:
            print("Someone closed the program") 
        print(link)
        html2 = requests.get(link)
        soup2= BeautifulSoup(html2.content, 'html.parser')
        if soup2.find('h1', {'class':'cui-heading-2--secondary vehicle-info__title'}): # if container for brand of car exists, create a variable brand and proceed to parse the rest of the information, else, skip to the next link
            brand = soup2.find('h1', {'class':'cui-heading-2--secondary vehicle-info__title'}).text.strip()
            print(brand)
            if soup2.find('span', {'class':'vehicle-info__price-display vehicle-info__price-display--dealer cui-heading-2'}): # ditto, but for price
                price = soup2.find('span', {'class':'vehicle-info__price-display vehicle-info__price-display--dealer cui-heading-2'}).text.replace(',','').replace('$','').strip()
                dict1 = {'brand': brand, 'price':price}
                basic_items = soup2.findAll('li',{'class':'vdp-details-basics__item'}) # the 'basic_items container' has information on upto 10 potential variables, however, some of these variables can be missing from one car to another
                for item in basic_items:
                    item = item.text.replace('\n','').strip()
                    list_item = item.split(':') # this splits the parsed information to create key and value for the dictionary below
                    # key is created here from the index no. 0 of the list, replacing space with '_' and lowering case = value is created here out of index number [1] of the above list. The 'if' statement checks for numeric strings to convert them to integer type objects, else keep it as it is.
                    dict1[list_item[0].strip().replace(' ','_').lower()]= int(list_item[1].strip().replace(',','')) if list_item[1].strip().replace(',','').isnumeric() else list_item[1].strip()
                print(dict1)
                all_cars.append(dict1)
                time.sleep(1)
            
            else:
                continue
        else:
            continue
        
    return all_cars

### Using the functions

In [5]:
# this was carried out in two batches a 1000 links then 2100
ns = list(range(0,11))
urlpt1 = 'https://www.cars.com/for-sale/searchresults.action/?dealerType=localOnly&fuelTypeId=38745,31763,31764&page='
urlpt2 = '&perPage=100&rd=10&searchSource=PAGINATION&slrTypeId=28878&sort=relevance&stkTypId=28881&yrId=51683,56007,58487,30031936,35797618&zc=10004'
url_1000_cars = link_list_v0(urlpt1,urlpt2, ns)

In [ ]:
ns1 = list(range(15,26))
urlpt1 = 'https://www.cars.com/for-sale/searchresults.action/?dealerType=localOnly&fuelTypeId=38745,31763,31764&page='
urlpt2 = '&perPage=100&rd=10&searchSource=PAGINATION&slrTypeId=28878&sort=relevance&stkTypId=28881&yrId=51683,56007,58487,30031936,35797618&zc=10004'
url_2100_cars = link_list_v0(urlpt1,urlpt2, ns)

In [ ]:
car_data_1000 = car_parser_v2(url_1000_cars)

In [21]:
car_data_2000 = car_parser_v2(url_2100_cars)

https://www.cars.com/vehicledetail/detail/782314103/overview/
2018 Ford F-150 XLT
{'brand': '2018 Ford F-150 XLT', 'price': '31995', 'fuel_type': 'Gasoline', 'exterior_color': 'Lead Foot', 'city_mpg': 18, 'interior_color': 'Dark Earth Gray', 'highway_mpg': 23, 'stock': 'KE79637A', 'drivetrain': '4WD', 'transmission': 'Automatic', 'engine': '3.5L V6 24V PDI DOHC Twin Turbo', 'vin': '1FTEW1EG9JKE79637', 'mileage': 26796}
https://www.cars.com/vehicledetail/detail/782314148/overview/
2016 Nissan Murano S
{'brand': '2016 Nissan Murano S', 'price': '20995', 'fuel_type': 'Gasoline', 'exterior_color': 'Arctic Blue Metallic', 'city_mpg': 21, 'interior_color': 'Black', 'highway_mpg': 28, 'stock': 'N152270A', 'drivetrain': 'AWD', 'transmission': 'Automatic CVT', 'engine': '3.5L V6 24V MPFI DOHC', 'vin': '5N1AZ2MHXGN152270', 'mileage': 14197}
https://www.cars.com/vehicledetail/detail/782656904/overview/
2016 Dodge Charger SXT
{'brand': '2016 Dodge Charger SXT', 'price': '23495', 'fuel_type': 'Gaso

https://www.cars.com/vehicledetail/detail/781912745/overview/
2016 Mercedes-Benz GLA 250 4MATIC
{'brand': '2016 Mercedes-Benz GLA 250 4MATIC', 'price': '26826', 'fuel_type': 'Gasoline', 'exterior_color': 'Mountain Gray Metallic', 'city_mpg': 24, 'interior_color': 'Black', 'highway_mpg': 32, 'stock': 'U13351', 'drivetrain': '4MATIC', 'transmission': '7-Speed Automatic', 'engine': '2.0L I4 DI Turbocharged', 'vin': 'WDCTG4GB5GJ246628', 'mileage': 35952}
https://www.cars.com/vehicledetail/detail/780672884/overview/
2017 Mercedes-Benz E 300 4MATIC
{'brand': '2017 Mercedes-Benz E 300 4MATIC', 'price': '40372', 'fuel_type': 'Gasoline', 'exterior_color': 'Iridium Silver Metallic', 'city_mpg': 22, 'interior_color': 'Black', 'highway_mpg': 29, 'stock': 'U13307', 'drivetrain': '4MATIC', 'transmission': '9-Speed Automatic', 'engine': 'I4', 'vin': 'WDDZF4KBXHA037466', 'mileage': 20169}
https://www.cars.com/vehicledetail/detail/780672880/overview/
2016 Mercedes-Benz CLA 250 4MATIC
{'brand': '2016 Me

2016 Honda Accord LX
{'brand': '2016 Honda Accord LX', 'price': '12981', 'fuel_type': 'Gasoline', 'exterior_color': 'Bronze', 'city_mpg': 23, 'interior_color': 'Black', 'highway_mpg': 34, 'stock': 1412, 'drivetrain': 'FWD', 'transmission': 'Automatic', 'engine': '2.4L L4 DOHC 16V', 'vin': '1HGCR2F39GA202261', 'mileage': 52887}
https://www.cars.com/vehicledetail/detail/781587000/overview/
2014 Mercedes-Benz C 300 4MATIC Sport
{'brand': '2014 Mercedes-Benz C 300 4MATIC Sport', 'price': '14981', 'fuel_type': 'Gasoline', 'exterior_color': 'Red', 'city_mpg': 20, 'interior_color': 'Black', 'highway_mpg': 27, 'stock': 1434, 'drivetrain': 'AWD', 'transmission': 'Automatic', 'engine': '3.5L V6 DOHC 16V', 'vin': 'WDDGF8AB0EA949213', 'mileage': 52768}
https://www.cars.com/vehicledetail/detail/777256632/overview/
2016 Lexus NX 200t F Sport
{'brand': '2016 Lexus NX 200t F Sport', 'price': '25149', 'fuel_type': 'Gasoline', 'exterior_color': 'Red', 'city_mpg': 22, 'highway_mpg': 28, 'stock': 'G206963

https://www.cars.com/vehicledetail/detail/774744526/overview/
2014 Ford Focus SE
{'brand': '2014 Ford Focus SE', 'price': '8295', 'fuel_type': 'Gasoline', 'exterior_color': 'Gray', 'city_mpg': 26, 'interior_color': 'Charcoal Black', 'highway_mpg': 36, 'stock': 'BC10617', 'drivetrain': 'FWD', 'transmission': 'Manual', 'engine': '2.0 L', 'vin': '1FADP3F23EL267975', 'mileage': 26456}
https://www.cars.com/vehicledetail/detail/775850513/overview/
2016 Audi A6 2.0T Premium
{'brand': '2016 Audi A6 2.0T Premium', 'price': '24895', 'fuel_type': 'Gasoline', 'city_mpg': 24, 'highway_mpg': 34, 'stock': '# M4337', 'vin': 'WAUFFAFC5GN097884', 'mileage': 34995}
https://www.cars.com/vehicledetail/detail/782284901/overview/
Certified 2016 Acura TLX V6
{'brand': 'Certified 2016 Acura TLX V6', 'price': '21078', 'fuel_type': 'Gasoline', 'exterior_color': 'Bellanova White Pearl', 'city_mpg': 21, 'interior_color': 'Espresso', 'highway_mpg': 34, 'stock': 'U24227', 'drivetrain': 'FWD', 'transmission': '9-Spee

https://www.cars.com/vehicledetail/detail/780585208/overview/
2016 Honda Civic EX
{'brand': '2016 Honda Civic EX', 'price': '14850', 'fuel_type': 'Gasoline', 'exterior_color': 'Black', 'city_mpg': 31, 'interior_color': 'Black / Gray', 'highway_mpg': 41, 'stock': 'B11567', 'drivetrain': 'FWD', 'transmission': 'Automatic CVT', 'engine': '2.0L I4 16V MPFI DOHC', 'vin': '19XFC2F74GE226691', 'mileage': 24648}
https://www.cars.com/vehicledetail/detail/780704848/overview/
2017 Acura RDX Technology & AcuraWatch Plus Packages
{'brand': '2017 Acura RDX Technology & AcuraWatch Plus Packages', 'price': '23800', 'fuel_type': 'Gasoline', 'exterior_color': 'Blue', 'city_mpg': 19, 'interior_color': 'Black', 'highway_mpg': 27, 'stock': 'HL006325', 'drivetrain': 'AWD', 'transmission': 'Automatic', 'engine': '3.5L V6 279hp 252ft. lbs.', 'vin': '5J8TB4H59HL006325', 'mileage': 47219}
https://www.cars.com/vehicledetail/detail/779071398/overview/
2016 BMW X3 xDrive28i
{'brand': '2016 BMW X3 xDrive28i', 'pric

...

https://www.cars.com/vehicledetail/detail/783971172/overview/
2017 Acura MDX 3.5L w/Technology Package
{'brand': '2017 Acura MDX 3.5L w/Technology Package', 'price': '34000', 'fuel_type': 'Gasoline', 'exterior_color': 'Modern Steel Metallic', 'city_mpg': 19, 'interior_color': 'Ebony', 'highway_mpg': 27, 'stock': 'W11327L', 'drivetrain': 'AWD', 'transmission': '9-Speed Automatic', 'engine': '3.5L V6 24V GDI SOHC', 'vin': '5FRYD4H58HB035461', 'mileage': 16387}
https://www.cars.com/vehicledetail/detail/783033708/overview/
2016 Acura TLX FWD
{'brand': '2016 Acura TLX FWD', 'price': '18250', 'fuel_type': 'Gasoline', 'exterior_color': 'Bellanova White Pearl', 'stock': 'U24288', 'drivetrain': 'FWD', 'transmission': '8-Speed Automatic with Auto-Shift', 'engine': '2.4L I4 16V GDI DOHC', 'vin': '19UUB1F37GA015877', 'mileage': 33071}
https://www.cars.com/vehicledetail/detail/783970779/overview/
2018 Acura MDX 3.5L
{'brand': '2018 Acura MDX 3.5L', 'price': '33500', 'fuel_type': 'Gasoline', 'exteri

2018 Honda Ridgeline RTL-E
{'brand': '2018 Honda Ridgeline RTL-E', 'price': '34295', 'fuel_type': 'Gasoline', 'exterior_color': 'Black', 'city_mpg': 18, 'highway_mpg': 25, 'stock': 'B11817T', 'drivetrain': 'AWD', 'transmission': '6-Speed Automatic', 'engine': '3.5L V6 24V GDI SOHC', 'vin': '5FPYK3F76JB013994', 'mileage': 14338}
https://www.cars.com/vehicledetail/detail/782859893/overview/
2017 Honda Accord EX-L
{'brand': '2017 Honda Accord EX-L', 'price': '21093', 'fuel_type': 'Gasoline', 'exterior_color': 'Black', 'city_mpg': 27, 'interior_color': 'Black', 'highway_mpg': 36, 'stock': 'B11879', 'drivetrain': 'FWD', 'transmission': 'Automatic CVT', 'engine': '2.4L I4 16V GDI DOHC', 'vin': '1HGCR2F85HA015886', 'mileage': 11160}
https://www.cars.com/vehicledetail/detail/783029998/overview/
2016 Honda CR-V Touring
{'brand': '2016 Honda CR-V Touring', 'price': '21866', 'fuel_type': 'Gasoline', 'exterior_color': 'Gray', 'city_mpg': 25, 'interior_color': 'Gray', 'highway_mpg': 31, 'stock': 'B

https://www.cars.com/vehicledetail/detail/769967757/overview/
2016 GMC Acadia SLE-1
{'brand': '2016 GMC Acadia SLE-1', 'price': '11995', 'fuel_type': 'Gasoline', 'exterior_color': 'Gray', 'city_mpg': 15, 'highway_mpg': 22, 'stock': 10856, 'drivetrain': 'AWD', 'transmission': '6-Speed Automatic', 'engine': '3.6L V6 24V GDI DOHC', 'vin': '1GKKVNED3GJ220504', 'mileage': 104951}
https://www.cars.com/vehicledetail/detail/779577830/overview/
2015 Chevrolet Malibu 1LT
{'brand': '2015 Chevrolet Malibu 1LT', 'price': '6995', 'fuel_type': 'Gasoline', 'exterior_color': 'Gold', 'city_mpg': 25, 'highway_mpg': 36, 'stock': 10909, 'drivetrain': 'FWD', 'transmission': '6-Speed Automatic', 'engine': '2.5L I4 16V GDI DOHC', 'vin': '1G11C5SL3FF342332', 'mileage': 90096}
https://www.cars.com/vehicledetail/detail/778359359/overview/
2014 Acura RDX Base
{'brand': '2014 Acura RDX Base', 'price': '11995', 'fuel_type': 'Gasoline', 'exterior_color': 'Gray', 'city_mpg': 19, 'highway_mpg': 27, 'stock': 10991, 'dr

https://www.cars.com/vehicledetail/detail/780295881/overview/
2015 Nissan Pathfinder SL
{'brand': '2015 Nissan Pathfinder SL', 'price': '12995', 'fuel_type': 'Gasoline', 'exterior_color': 'White', 'city_mpg': 20, 'interior_color': 'Beige', 'highway_mpg': 27, 'stock': 605090, 'drivetrain': 'AWD', 'transmission': 'Automatic', 'engine': '3.5L V6 DOHC 24V', 'vin': '5N1AR2MM7FC605090', 'mileage': 125104}
https://www.cars.com/vehicledetail/detail/783283442/overview/
2015 Nissan Altima 2.5 SV
{'brand': '2015 Nissan Altima 2.5 SV', 'price': '9995', 'fuel_type': 'Gasoline', 'exterior_color': 'White', 'city_mpg': 27, 'interior_color': 'Black', 'highway_mpg': 38, 'stock': 861581, 'drivetrain': 'FWD', 'transmission': 'Automatic', 'engine': '2.5L L4 DOHC 16V', 'vin': '1N4AL3AP3FN861571', 'mileage': 60026}
https://www.cars.com/vehicledetail/detail/783283451/overview/
2014 Lexus GS 350 Base
{'brand': '2014 Lexus GS 350 Base', 'price': '16995', 'fuel_type': 'Gasoline', 'exterior_color': 'Black', 'city

https://www.cars.com/vehicledetail/detail/781300734/overview/
2014 Hyundai Elantra Limited
{'brand': '2014 Hyundai Elantra Limited', 'price': '6999', 'fuel_type': 'Gasoline', 'exterior_color': 'Venetian Red', 'city_mpg': 27, 'interior_color': 'Beige', 'highway_mpg': 37, 'stock': 501181, 'drivetrain': 'FWD', 'transmission': 'Automatic', 'engine': '4 Cylinder Engine', 'vin': '5NPDH4AE3EH501181', 'mileage': 65420}
https://www.cars.com/vehicledetail/detail/777704944/overview/
2015 Audi A4 2.0T Premium Plus
{'brand': '2015 Audi A4 2.0T Premium Plus', 'price': '16995', 'fuel_type': 'Gasoline', 'exterior_color': 'Red', 'interior_color': 'BLACK', 'stock': 13437, 'transmission': 'Automatic', 'engine': '2.0L L4 DOHC 16V TURBO', 'vin': 'WAUFFAFL4FA069676', 'mileage': 46499}
https://www.cars.com/vehicledetail/detail/783282680/overview/
2015 Mercedes-Benz S 63 AMG
{'brand': '2015 Mercedes-Benz S 63 AMG', 'price': '67995', 'fuel_type': 'Gasoline', 'exterior_color': 'Black', 'city_mpg': 15, 'interior

In [ ]:
# Save cars_data_full as a json file
with open('cars_data_full.json', 'w') as jfile:
    json.dump(car_data_addi_2000, jfile)

In [24]:
# Save cars_data_full as a json file
with open('cars_data_2100.json', 'w') as jfile:
    json.dump(car_data_addi_2000, jfile)